<a href="https://colab.research.google.com/github/vignesh99/Speech-Technology-CS6300/blob/master/E2E_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
                                    #Get data files onto root
%cd
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/TIMIT\ E2E/timit.zip /root
!unzip /root/timit.zip -d /root
!cp -avr /content/gdrive/My\ Drive/TIMIT\ E2E/Wordset\ files/* /root
!cp -avr /content/gdrive/My\ Drive/TIMIT\ E2E/Wordceps\ files/* /root
!ls

In [2]:
                                    #Import libraries
from pylab import *
import os
from google.colab import files
!pip install pickle-mixin           #Use pkl to store model
from pickle import dump
from pickle import load      
!pip install SoundFile
import soundfile as sf              #For .wav to data
from scipy.io import wavfile
!pip install python_speech_features
from python_speech_features import mfcc
import pandas as pd
import csv

In [3]:
#words = load(open('words.pkl', 'rb'))
#inds = load(open('inds.pkl', 'rb'))
reps = load(open('reps.pkl', 'rb'))
wordset = load(open('wordset.pkl', 'rb'))

Nreps = 250							#Number of repititions of each word

inds = np.where(reps >= Nreps)[0]
words = wordset[inds]               #Choose only 36-most-repeated words
freqs = reps[inds]                  #Frequency at which these words occur
print(len(inds))
print(words)
print(freqs)

27
['a' 'all' 'an' 'and' 'ask' 'carry' 'dark' "don't" 'greasy' 'had' 'in'
 'is' 'like' 'me' 'of' 'oily' 'rag' 'she' 'suit' 'that' 'the' 'to' 'wash'
 'water' 'year' 'you' 'your']
[ 866  543  571  481  464  463  465  488  462  526  944  399  511  505
  454  470  470  571  462  611 1602 1018  469  479  472  271  565]


In [4]:
def mostrepeatedwords(Nrep=250) :
    allwords = []
                                    #Go through all folders and consolidate the words
    for subdir, dirs, files in os.walk("timit_train"):
        dirname = subdir.split(os.path.sep)[-1]
        print('Directory:', dirname)
        for f in files:
            if f.endswith('.txt'):
                                    #Get words of each sentence from .txt file
                !cp -avr /root/timit_train/$dirname/$f /root
                words = np.genfromtxt(f,delimiter=' ',dtype=str)
                words = words[2:]
                words[-1] = words[-1][:-1]
                                    #Put all the words in a list
                allwords += list(map(str.lower,list(words)))
                !rm $f
                                    #Get the unique set of words and their counts
    wordunique,wordcount = np.unique(allwords,return_counts=True)
                                    #Choose only those words with significant repitions
    indrep = np.where(wordcount >= Nrep)[0]
    #print(len(indrep))
    wordclass = wordunique[indrep]
    return wordclass,wordunique,wordcount,indrep

In [5]:
# words,wordset,reps,inds = mostrepeatedwords()
# words

In [6]:
                                    #Dump words model file
# dump(words, open('words.pkl', 'wb'))
# dump(inds, open('inds.pkl', 'wb'))
# dump(reps, open('reps.pkl', 'wb'))
# dump(wordset, open('wordset.pkl', 'wb'))

In [7]:
                                    #Download model files
# files.download("/root/words.pkl")
# files.download("/root/inds.pkl")
# files.download("/root/reps.pkl")
# files.download("/root/wordset.pkl")

In [8]:
                                    #Getting the mfcc features for different words
def wavtoMfcc(words,Nmfcc=38) :
    count = 0                       #Initialize required lists
    timit_audio = []
    new_timit_audio = []
    timitceps = []
    wordloop = []
    lenframes = []
    for subdir, dirs, files in os.walk('timit_train'):
        dirname = subdir.split(os.path.sep)[-1]
        for f in files:
            if f.endswith('.wav'):
                count += 1
                print(os.path.join(subdir, f), ', name:', f, ', Count:', count)
                                        #Read data from the .wav file
                data, fs = sf.read(os.path.join(subdir, f))
                timit_audio.append(data)
                fword = f[:-4]+".wrd"
                                        #Check word files for most repeated words
                print(os.path.join(subdir, fword), ', name:', fword, ', Count:', count)
                word_file = np.array(pd.read_csv(os.path.join(subdir, fword), delimiter = ' ',header = None))
                new_data = []
                for w in np.array(word_file[:,2]):
                    if w in words:
                        a = np.where(word_file[:,2] == w)
                        start = word_file[a[0],0][0]
                        end = word_file[a[0],1][0]
                        if(start != end):   #For removing corner case error
                                            #Collect the .wav samples of required slices
                            new_data = data[start:end]
                            new_timit_audio.append(new_data)
                                            #Convert them to mfcc
                            ceps = mfcc(new_data, fs, numcep=Nmfcc,winlen=0.008,winstep=0.004,nfilt=76)
                            timitceps.append(ceps)
                                            #Put the words in order of the mfcc features
                            wordloop.append(w)
                            lenframes.append(len(ceps))
    
    timitceps = array(timitceps)
    wordloop = array(wordloop)
    lenframes = array(lenframes) 
    maxframes = lenframes.max()             #Obtain maximum size of frames

    return timitceps,wordloop,lenframes,maxframes,new_timit_audio                                           

In [ ]:
# timitceps,wordloop,lenframes,maxframes,new_timit_audio = wavtoMfcc(words)

In [13]:
                                    #Dump processed files
# dump(timitceps, open('timitceps.pkl', 'wb'))
# dump(wordloop, open('wordloop.pkl', 'wb'))
# dump(lenframes, open('lenframes.pkl', 'wb'))
# dump(new_timit_audio, open('new_timit_audio.pkl', 'wb'))

In [14]:
                                    #Download model files
# files.download("/root/timitceps.pkl")
# files.download("/root/wordloop.pkl")
# files.download("/root/lenframes.pkl")
# files.download("/root/new_timit_audio.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
                                    #Load the required files
timitceps = load(open('timitceps.pkl', 'rb'))
wordloop = load(open('wordloop.pkl', 'rb'))
lenframes = load(open('lenframes.pkl', 'rb'))
# new_timit_audio = load(open('new_timit_audio.pkl', 'rb'))

In [ ]:
def mfcctoInput(words,freqs,timitceps,wordloop,lenframes,maxframes=64,Nmfcc=38) :
    for i in range(0,len(words)) :
                                                #Obtain the particular word's mfcc features
        indcep = np.where(wordloop==words[i])[0]
        indlen = lenframes[indcep]
                                                #Create a specific matrix based on the occurances
        wordcepmatrix = np.zeros((freqs[i],maxframes,Nmfcc))
        if freqs[i] == len(indcep) :             #Check if occurance match
            for j in range(0,len(indcep)) :
                wordcepmatrix[j][:indlen[j]] = timitceps[indcep[j]]

        else :
            print("ERROR : Frequencies don't match with index length")
            return None

In [ ]:
print(shape(timitceps))
np.unique(wordloop,return_counts=True)

(15650,)


(array(['a', 'all', 'an', 'and', 'ask', 'carry', 'dark', "don't", 'greasy',
        'had', 'in', 'is', 'like', 'me', 'of', 'oily', 'rag', 'she',
        'suit', 'that', 'the', 'to', 'wash', 'water', 'year', 'you',
        'your'], dtype='<U6'),
 array([ 865,  545,  571,  492,  464,  463,  473,  488,  462,  526,  947,
         401,  518,  517,  453,  470,  470,  572,  462,  612, 1603, 1016,
         469,  479,  473,  274,  565]))

In [ ]:
a = np.where(wordloop=="carry")[0]
shape(timitceps[a[5]])
#wordloop[2]

(73, 38)

In [12]:
wordloop[np.where(lenframes==lenframes.min())[0]]
#wavfile.write('test.wav', 16000, new_timit_audio[15545])

array(['to', 'a'], dtype='<U6')

In [10]:
print(lenframes.max())
lenframes.min()

162


3

In [ ]:
print(words)
print(freqs)

['a' 'all' 'an' 'and' 'are' 'ask' 'be' 'carry' 'dark' "don't" 'for'
 'greasy' 'had' 'he' 'his' 'in' 'is' 'like' 'me' 'of' 'oily' 'on' 'rag'
 'she' 'suit' 'that' 'the' 'to' 'was' 'wash' 'water' 'we' 'with' 'year'
 'you' 'your']
[ 866  543  571  481  237  464  175  463  465  488  216  462  526  233
  189  944  399  511  505  454  470  164  470  571  462  611 1602 1018
  235  469  479  154  187  472  271  565]


In [ ]:
files.download("test.wav")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>